In [3]:
import json
import os
from tqdm import tqdm

from openai import OpenAI
from mcq_prompts import *

#####################################################
FILE_PATH = "../files/prompt_direct+indirect_0-100.json" # .json
OUT_DIR = "../files" # directory to output
ANNOT_IMG_DIR = "../../data/annotated_images"
#####################################################

os.makedirs(OUT_DIR, exist_ok=True)

In [5]:
file = open("../../api.txt", "r")
content = file.read()
client = OpenAI(
    api_key=content
)
with open(FILE_PATH, 'r') as f:
    data = json.load(f)

In [6]:
import random
random.seed(0)
num = len(data)
print(num)
# 1. generate answer
for idx, d in enumerate(tqdm(data)):
    p = d['solution'][1:-1].split(',')[0]
    solution_action = d['solution'].split(',')[1][1:]
    solution_entity = d['solution'][1:-1].split(',')[-1][1:]
    direct = d['prompt']
    indirect = d['indirect']
    answer = gen_answer(client, solution_action, solution_entity, direct, indirect,p)
    data[idx]['correct']  = answer
    
# 2. generate incorrect choices
for idx, d in enumerate(tqdm(data)):
    
    # info
    image_name = d['name'][0]
    entities = d['entities']
    p = d['solution'][1:-1].split(',')[0]
    solution_action = d['solution'].split(',')[1][1:]
    solution_entity = d['solution'][1:-1].split(',')[-1][1:]
    direct = d['prompt']
    indirect = d['indirect']
    answer = d['correct']
    caption = d['caption']
    
    r1, r2, r3 = random.sample(range(0, num), 3)
    icls  = [data[r1]['correct'], data[r2]['correct'], data[r3]['correct']]
    choice_list = ['A', 'B', 'C', 'D', 'E']
    random.shuffle(choice_list)

    if image_name:
        annotated_image_name = image_name + "_annot.jpg"
        annotated_image_path = os.path.join(ANNOT_IMG_DIR, annotated_image_name)
        if os.path.exists(annotated_image_path):
            encoded_image = encode_image(annotated_image_path)
            mcq_wrong_entity = gen_wrong_entity(client, encoded_image, indirect, answer)
        else:
            mcq_wrong_entity = ''
            print(f'Image not found : {image_name}')
    fake_cap = fake_caption(client, direct, indirect ,p)
    mcq_fake = gen_fake_answer(client, indirect, icls, fake_cap, answer, p)
    sum_indirect = summarize_indirect(client, indirect)
    # mcq_surface1 = gen_surface_understanding1(client, caption, sum_indirect, p, direct)
    mcq_surface2 = gen_surface_understanding2(client, direct, indirect, caption, p)

    data[idx]['incorrect_fake_scene']  = mcq_fake
    data[idx]['incorrect_wrong_entity'] = mcq_wrong_entity
    # data[idx]['incorrect_surface1'] = mcq_surface1
    data[idx]['incorrect_surface'] = mcq_surface2
    data[idx]['ordering'] = choice_list
    
with open(os.path.join(OUT_DIR,'prompt_all_0-100_updated_1027.json'), 'w') as f:
    json.dump(data, f, indent=4)
    

97


  0%|          | 0/97 [00:00<?, ?it/s]

100%|██████████| 97/97 [13:00<00:00,  8.05s/it]
